RAG model


In [ ]:
import os

if "COLAB_GPU" in os.environ:
    !pip install -U torch -qq > /dev/null 2>&1
    !pip install PyMuPDF -qq > /dev/null 2>&1
    !pip install pdfplumber -qq > /dev/null 2>&1
    !pip install python-docx -qq > /dev/null 2>&1
    !pip install docxtxt -qq > /dev/null 2>
    !pip install tqdm -qq > /dev/null 2>&1
    !pip install sentence-transformers -qq > /dev/null 2>&1
    !pip install accelerate -qq > /dev/null 2>&1
    !pip install bitsandbytes -qq > /dev/null 2>&1
    !pip install flash-attn --no-build-isolation -qq > /dev/null 2>&1


/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `pip install docxtxt -qq > /dev/null 2>'


In [ ]:
from google.colab import files
from tqdm.auto import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
import fitz  # PyMuPDF for PDF processing

# Global variable to store the uploaded file name
uploaded_file_name = None
pages_and_texts = []

# Function to handle file upload
def upload_pdf():
    global uploaded_file_name
    print("Please upload your PDF file:")
    uploaded = files.upload()
    for file_name in uploaded.keys():
        if file_name.lower().endswith('.pdf'):
            print(f"File '{file_name}' uploaded successfully.")
            uploaded_file_name = file_name
        else:
            print(f"'{file_name}' is not a PDF file. Skipping...")

# Function to clean extracted text
def text_formatter(text: str) -> str:
    return text.replace("\n", " ").strip()

# Function to process PDF and extract text
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    try:
        doc = fitz.open(pdf_path)

        for page_number, page in tqdm(enumerate(doc, start=1), desc="Reading pages"):
            text = page.get_text()
            if text.strip():
                pages_and_texts.append({
                    "page_number": page_number,
                    "text": text_formatter(text)
                })
        doc.close()
        return pages_and_texts
    except Exception as e:
        print(f"An error occurred while reading the PDF: {e}")
        return []

# Function to handle button click
def on_button_click(b):
    clear_output()  # Clear the previous output
    upload_pdf()
    if uploaded_file_name:
        print(f"Processing the uploaded file: {uploaded_file_name}")
        pages_and_texts = open_and_read_pdf(uploaded_file_name)
        if pages_and_texts:
            print(f"Successfully processed {len(pages_and_texts)} pages.")

            # Sample 3 random pages
            num_samples = min(3, len(pages_and_texts))
            sampled_pages = random.sample(pages_and_texts, k=num_samples)
            print("\nRandom Samples:")
            for page in sampled_pages:
                print(f"Page {page['page_number']}: {page['text'][:200]}...\n")  # First 200 chars
        else:
            print("No text extracted from the PDF.")
    else:
        print("No valid PDF file was uploaded.")

# Create the upload button with an icon
upload_button = widgets.Button(
    description="Upload PDF",
    icon="upload",
    button_style="success"  # 'success' makes the button green
)

# Assign the button click event
upload_button.on_click(on_button_click)

# Display the button
display(upload_button)


Please upload your PDF file:


In [ ]:
import random
random.sample(pages_and_texts, k=3)

ValueError: Sample larger than population or is negative

In [ ]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
df.head()

In [ ]:
df.describe().round(2)

# Further text processing (splitting pages into sentences)

In [ ]:
from spacy.lang.en import English
nlp = English()
nlp.add_pipe("sentencizer")

In [ ]:
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    item["page_sentence_count_spacy"] = len(item["sentences"])

In [ ]:
random.sample(pages_and_texts, k=1)

In [ ]:
print(item["page_sentence_count_spacy"])

# Chunking our sentences together

In [ ]:
num_sentence_chunk_size = 10
def split_list(input_list: list,
               slice_size: int) -> list[list[str]]:

    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

In [ ]:
random.sample(pages_and_texts, k=1)

# Splitting each chunk into its own item

In [ ]:
import pandas as pd
import re
from tqdm.auto import tqdm

# Ensure pages_and_chunks is properly initialized
pages_and_chunks = []

# Populate pages_and_chunks from pages_and_texts
for item in tqdm(pages_and_texts, desc="Processing pages"):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        # Join sentences into a chunk
        joined_sentence_chunk = ". ".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Add metadata
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len(joined_sentence_chunk.split(" "))
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4  # Approx token count

        pages_and_chunks.append(chunk_dict)

# Convert to DataFrame
df = pd.DataFrame(pages_and_chunks)

# Check for missing columns
if "chunk_token_count" not in df.columns:
    print("Adding 'chunk_token_count' to the DataFrame...")
    df["chunk_token_count"] = df["sentence_chunk"].apply(lambda x: len(x) / 4 if isinstance(x, str) else 0)

# Check the number of rows and columns
print(f"DataFrame has {len(df)} rows and {len(df.columns)} columns.")
print(df.head())

# Filter and sample
min_token_length = 10
filtered_df = df[df["chunk_token_count"] <= min_token_length]

if not filtered_df.empty:
    for _, row in filtered_df.sample(1).iterrows():
        print(f'Chunk token count: {row["chunk_token_count"]} | Text: {row["sentence_chunk"]}')
else:
    print("No chunks meet the condition. Sampling is not possible.")


In [ ]:
random.sample(pages_and_chunks, k=1)

In [ ]:
chunk_dict


In [ ]:
# Get stats about our chunks
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,83.00,83.00,83.00,83.00
mean,13.98,706.88,130.87,176.72
std,8.14,358.62,68.07,89.65
min,1.00,20.00,2.00,5.00
25%,7.00,450.00,87.00,112.50
50%,14.00,681.00,123.00,170.25
75%,20.00,885.00,157.50,221.25
max,27.00,1638.00,306.00,409.50


In [ ]:

import random

if len(pages_and_chunks) > 0:
    random_sample = random.sample(pages_and_chunks, k=1)
    print(random_sample)
else:
    print("The list 'pages_and_chunks' is empty, so sampling is not possible.")

[{'page_number': 25, 'sentence_chunk': 'The net power (P ) of the lenses placed in contact is given by the algebraic sum of the individual powers P1, P2, P3, …  as P = P1 + P2 + P3 + … The use of powers, instead of focal lengths, for lenses is quite convenient for opticians.. During eye-testing, an optician puts several different combinations of corrective lenses of known power, in contact, inside the testing spectacles’ frame.. The optician calculates the power of the lens required by simple algebraic addition.. For example, a combination of two lenses of power + 2.0 D and + 0.25 D is equivalent to a single lens of power + 2.25 D. The simple additive property of the powers of lenses can be used to design lens systems to minimise certain defects in images produced by a single lens.. Such a lens system, consisting of several lenses, in contact, is commonly used in the design of lenses of camera, microscopes and telescopes.. Q U E S T I O N S ?. 1.. Define 1 dioptre of power of a lens.. 

In [ ]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,83.00,83.00,83.00,83.00
mean,13.98,706.88,130.87,176.72
std,8.14,358.62,68.07,89.65
min,1.00,20.00,2.00,5.00
25%,7.00,450.00,87.00,112.50
50%,14.00,681.00,123.00,170.25
75%,20.00,885.00,157.50,221.25
max,27.00,1638.00,306.00,409.50


In [ ]:
print("Columns in DataFrame:", df.columns)

Columns in DataFrame: Index(['page_number', 'sentence_chunk', 'chunk_char_count', 'chunk_word_count',
       'chunk_token_count'],
      dtype='object')


In [ ]:
min_token_length = 10
filtered_df = df[df["chunk_token_count"] <= min_token_length]

if not filtered_df.empty:
    # Sample one row and print
    for _, row in filtered_df.sample(1).iterrows():
        print(f'Chunk token count: {row["chunk_token_count"]} | Text: {row["sentence_chunk"]}')
else:
    print("No chunks meet the condition. Sampling is not possible.")

Chunk token count: 5.0 | Text: Rationalised 2023-24


In [ ]:
print(df.columns)
print(df.head())

Index(['page_number', 'sentence_chunk', 'chunk_char_count', 'chunk_word_count',
       'chunk_token_count'],
      dtype='object')
   page_number                                     sentence_chunk  \
0            1  Science 134 Light – Reflection and Refraction ...   
1            1  By observing the common optical phenomena arou...   
2            1  We shall try to understand in this Chapter the...   
3            2  Light – Reflection and Refraction 135 Let us r...   
4            2  Activity 9.1 Activity 9.1 Activity 9.1 Activit...   

   chunk_char_count  chunk_word_count  chunk_token_count  
0               771               138             192.75  
1              1370               240             342.50  
2               452                75             113.00  
3               788               143             197.00  
4               424                78             106.00  


In [ ]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': 1,
  'sentence_chunk': 'Science 134 Light – Reflection and Refraction 9 CHAPTER W e see a variety of objects in the world around us.. However, we are unable to see anything in a dark room.. On lighting up the room, things become visible.. What makes things visible?. During the day, the sunlight helps us to see objects.. An object reflects light that falls on it.. This reflected light, when received by our eyes, enables us to see things.. We are able to see through a transparent medium as light is transmitted through it.. There are a number of common wonderful phenomena associated with light such as image formation by mirrors, the twinkling of stars, the beautiful colours of a rainbow, bending of light by a medium and so on.. A study of the properties of light helps us to explore them.',
  'chunk_char_count': 771,
  'chunk_word_count': 138,
  'chunk_token_count': 192.75},
 {'page_number': 1,
  'sentence_chunk': 'By observing the common optical phenomena around us, we ma

# Embedding our text chunks

In [ ]:
import os
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain_ollama import ChatOllama
from qdrant_client import QdrantClient
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
import streamlit as st

class ChatbotManager:
    def __init__(
        self,
        model_name: str = "BAAI/bge-small-en",
        device: str = "cpu",
        encode_kwargs: dict = {"normalize_embeddings": True},
        llm_model: str = "llama3.2:3b",
        llm_temperature: float = 0.7,
        qdrant_url: str = "http://localhost:6333",
        collection_name: str = "vector_db",
    ):
        """
        Initializes the ChatbotManager with embedding models, LLM, and vector store.

        Args:
            model_name (str): The HuggingFace model name for embeddings.
            device (str): The device to run the model on ('cpu' or 'cuda').
            encode_kwargs (dict): Additional keyword arguments for encoding.
            llm_model (str): The local LLM model name for ChatOllama.
            llm_temperature (float): Temperature setting for the LLM.
            qdrant_url (str): The URL for the Qdrant instance.
            collection_name (str): The name of the Qdrant collection.
        """
        self.model_name = model_name
        self.device = device
        self.encode_kwargs = encode_kwargs
        self.llm_model = llm_model
        self.llm_temperature = llm_temperature
        self.qdrant_url = qdrant_url
        self.collection_name = collection_name

        # Initialize Embeddings
        self.embeddings = HuggingFaceBgeEmbeddings(
            model_name=self.model_name,
            model_kwargs={"device": self.device},
            encode_kwargs=self.encode_kwargs,
        )

        # Initialize Local LLM
        self.llm = ChatOllama(
            model=self.llm_model,
            temperature=self.llm_temperature,
            # Add other parameters if needed
        )

        # Define the prompt template
        self.prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer. Answer must be detailed and well explained.
Helpful answer:
"""

        # Initialize Qdrant client
        self.client = QdrantClient(
            url=self.qdrant_url, prefer_grpc=False
        )

        # Initialize the Qdrant vector store
        self.db = Qdrant(
            client=self.client,
            embeddings=self.embeddings,
            collection_name=self.collection_name
        )

        # Initialize the prompt
        self.prompt = PromptTemplate(
            template=self.prompt_template,
            input_variables=['context', 'question']
        )

        # Initialize the retriever
        self.retriever = self.db.as_retriever(search_kwargs={"k": 1})

        # Define chain type kwargs
        self.chain_type_kwargs = {"prompt": self.prompt}

        # Initialize the RetrievalQA chain with return_source_documents=False
        self.qa = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.retriever,
            return_source_documents=False,  # Set to False to return only 'result'
            chain_type_kwargs=self.chain_type_kwargs,
            verbose=False
        )

    def get_response(self, query: str) -> str:
        """
        Processes the user's query and returns the chatbot's response.

        Args:
            query (str): The user's input question.

        Returns:
            str: The chatbot's response.
        """
        try:
            response = self.qa.run(query)
            return response  # 'response' is now a string containing only the 'result'
        except Exception as e:
            st.error(f"⚠️ An error occurred while processing your request: {e}")
            return "⚠️ Sorry, I couldn't process your request at the moment."

In [ ]:
import sys
import contextlib
from sentence_transformers import SentenceTransformer

# Initialize the embedding model
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2")

# Ensure pages_and_chunks is already populated from the first block
# Filter chunks with sufficient token length
min_token_length = 10
pages_and_chunks_over_min_token_len = [
    chunk for chunk in pages_and_chunks if chunk.get("chunk_token_count", 0) > min_token_length
]

# Extract sentences from the filtered chunks
sentences = [chunk["sentence_chunk"] for chunk in pages_and_chunks_over_min_token_len]

# Check if we have extracted any sentences
if len(sentences) == 0:
    print("No sentences available for embedding. Check the token counts or data input.")
else:
    print(f"Total sentences available for embedding: {len(sentences)}")

    # Suppress output during embedding generation
    with contextlib.redirect_stdout(sys.stderr):
        # Generate embeddings silently
        embeddings = embedding_model.encode(sentences)

    # Create a dictionary of sentences and their embeddings
    embeddings_dict = dict(zip(sentences, embeddings))

    print("Embeddings have been generated successfully.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Total sentences available for embedding: 82
Embeddings have been generated successfully.


In [ ]:
%%time

embedding_model.to("cpu")
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/82 [00:00<?, ?it/s]

CPU times: user 53.1 s, sys: 149 ms, total: 53.3 s
Wall time: 53.8 s


In [ ]:
%%time

# Send the model to the GPU
#embedding_model.to("cuda")

# Create embeddings one by one on the GPU
#for item in tqdm(pages_and_chunks_over_min_token_len):
    #item["embedding"] = embedding_model.encode(item["sentence_chunk"])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [ ]:
# Turn text chunks into a single list
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]



In [ ]:
%%time

# Embed all texts in batches
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32,
                                               convert_to_tensor=True) # optional to return embeddings as tensor instead of array

text_chunk_embeddings

CPU times: user 1min 6s, sys: 15.2 s, total: 1min 21s
Wall time: 1min 21s


tensor([[ 0.0233, -0.0809, -0.0105,  ...,  0.0510,  0.0651, -0.0124],
        [ 0.0730, -0.0411, -0.0385,  ...,  0.0323,  0.0583, -0.0051],
        [ 0.0516, -0.0899, -0.0241,  ...,  0.0146,  0.0342, -0.0059],
        ...,
        [ 0.0287, -0.1119, -0.0264,  ...,  0.0449,  0.0020, -0.0448],
        [ 0.0096, -0.1286, -0.0239,  ...,  0.0321, -0.0436, -0.0089],
        [-0.0016, -0.1048,  0.0007,  ...,  0.0397, -0.0464, -0.0313]])

In [ ]:
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [ ]:
# Import saved file and view
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,1,Science 134 Light – Reflection and Refraction ...,771,138,192.75,[ 2.33361125e-02 -8.08539838e-02 -1.05124330e-...
1,1,By observing the common optical phenomena arou...,1370,240,342.50,[ 7.29979798e-02 -4.11310084e-02 -3.85360457e-...
2,1,We shall try to understand in this Chapter the...,452,75,113.00,[ 5.15893437e-02 -8.99004862e-02 -2.40990035e-...
3,2,Light – Reflection and Refraction 135 Let us r...,788,143,197.00,[ 4.72141989e-02 -1.02055632e-01 -1.61858369e-...
4,2,Activity 9.1 Activity 9.1 Activity 9.1 Activit...,424,78,106.00,[-2.50257719e-02 -1.04841195e-01 -4.19099368e-...


In [ ]:

import random

import torch
import numpy as np
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

# Import texts and embedding df
text_chunks_and_embedding_df = pd.read_csv("text_chunks_and_embeddings_df.csv")

# Convert embedding column back to np.array (it got converted to string when it got saved to CSV)
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

# Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape



torch.Size([82, 768])

In [ ]:
text_chunks_and_embedding_df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,1,Science 134 Light – Reflection and Refraction ...,771,138,192.75,"[0.0233361125, -0.0808539838, -0.010512433, 0...."
1,1,By observing the common optical phenomena arou...,1370,240,342.50,"[0.0729979798, -0.0411310084, -0.0385360457, -..."
2,1,We shall try to understand in this Chapter the...,452,75,113.00,"[0.0515893437, -0.0899004862, -0.0240990035, 0..."
3,2,Light – Reflection and Refraction 135 Let us r...,788,143,197.00,"[0.0472141989, -0.102055632, -0.0161858369, 0...."
4,2,Activity 9.1 Activity 9.1 Activity 9.1 Activit...,424,78,106.00,"[-0.0250257719, -0.104841195, -0.0419099368, 0..."


In [ ]:
embeddings[0]

tensor([ 2.3336e-02, -8.0854e-02, -1.0512e-02,  6.9898e-03, -1.9910e-02,
        -3.4123e-02,  7.7079e-03,  1.8473e-02, -1.7201e-02, -1.1135e-02,
         3.0575e-02,  1.3320e-02, -1.1784e-02, -2.0296e-02,  5.4312e-02,
        -1.2401e-01,  1.9789e-02,  3.1776e-03, -7.8977e-02,  4.7794e-03,
        -1.1990e-02, -3.1031e-02,  3.3893e-03, -2.9779e-02,  5.3325e-03,
         4.6337e-03, -2.3862e-02, -4.7742e-02, -2.3317e-02, -1.4739e-02,
        -4.4208e-02, -3.2930e-02, -2.8425e-03, -3.1180e-02,  2.1153e-06,
        -3.4411e-02,  1.3462e-02,  4.3063e-02, -1.8509e-02, -5.2729e-03,
         3.4540e-02, -1.9696e-02,  3.5065e-02,  2.9814e-02, -7.8892e-03,
         3.8541e-02, -6.2468e-03,  5.7362e-02, -6.7404e-02, -9.2597e-03,
        -9.1738e-03,  1.1122e-02,  3.1569e-02,  7.0344e-03,  1.1361e-01,
         4.5621e-02, -1.8490e-02, -4.3137e-02, -1.0842e-02,  6.0217e-02,
        -4.1209e-02, -3.7518e-02,  2.1210e-02,  2.8717e-02,  3.1310e-02,
         6.5484e-03, -2.6784e-02,  4.2565e-02,  4.3

In [ ]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device=device) # choose the device to load the model to

In [ ]:
import torch
from sentence_transformers import util
from timeit import default_timer as timer  # Import the timer

# Query input
query = input("Enter your Question: ")
print(f"Query: {query}")

# Embed the query to the same numerical space as the text examples
query_embedding = embedding_model.encode(query, convert_to_tensor=True)

# Get similarity scores with the dot product
start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()

print(f"Time taken to get scores on {len(embeddings)} embeddings: {end_time - start_time:.5f} seconds.")

# Get the top-k results
top_k = 2
top_results_dot_product = torch.topk(dot_scores, k=top_k)

# Extract the indices and scores of the top results
top_indices = top_results_dot_product.indices.tolist()
top_scores = top_results_dot_product.values.tolist()

# Display the top results
print("\nTop Results:")
for i, (index, score) in enumerate(zip(top_indices, top_scores)):
    chunk = pages_and_chunks[index]

    # Safely get the text data
    text = chunk.get('sentence_chunk', "No text available")
    print(f"\nResult {i+1}:")
    print(f"Score: {score:.4f}")
    print(f"Text: {text}")


Enter your Question: convex lens
Query: convex lens
Time taken to get scores on 82 embeddings: 0.00030 seconds.

Top Results:

Result 1:
Score: 0.6227
Text: The distance of the principal focus from the optical centre of a lens is called its focal length.. The letter f is used to represent the focal length.. How can you find the focal length of a convex lens?. Recall the Activity 9.11.. In this Activity, the distance between the position of the lens and the position of the image of the Sun gives the approximate focal length of the lens.. Rationalised 2023-24

Result 2:
Score: 0.6116
Text: Such lenses diverge light rays as shown in Fig.. 9.12 (b).. Such lenses are also called diverging lenses.. A double concave lens is simply called a concave lens.. A lens, either a convex lens or a concave lens, has two spherical surfaces.. Each of these surfaces forms a part of a sphere.. The centres of these spheres are called centres of curvature of the lens.. The centre of curvature of a lens is usu

In [ ]:
larger_embeddings = torch.randn(100*embeddings.shape[0], 768).to(device)
print(f"Embeddings shape: {larger_embeddings.shape}")

# Perform dot product across 168,000 embeddings
start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=larger_embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(larger_embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

Embeddings shape: torch.Size([8200, 768])
Time take to get scores on 8200 embeddings: 0.00318 seconds.


In [ ]:
# Define helper function to print wrapped text
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [ ]:
print(f"Query: '{query}'\n")
print("Results:")

# Loop through scores and indices from torch.topk
for score, idx in zip(top_results_dot_product[0], top_results_dot_product[1]):
    print(f"Score: {score:.4f}")
    # Print relevant sentence chunk
    print("Text:")
    print_wrapped(pages_and_chunks[idx]["sentence_chunk"])

    # Get page number with a fallback if it doesn't exist
    page_number = pages_and_chunks[idx].get("page_number", "N/A")  # Replace 'page_number' with actual key if different
    print(f"Page number: {page_number}")
    print("\n")


Query: 'convex lens'

Results:
Score: 0.6227
Text:
The distance of the principal focus from the optical centre of a lens is called
its focal length.. The letter f is used to represent the focal length.. How can
you find the focal length of a convex lens?. Recall the Activity 9.11.. In this
Activity, the distance between the position of the lens and the position of the
image of the Sun gives the approximate focal length of the lens.. Rationalised
2023-24
Page number: 18


Score: 0.6116
Text:
Such lenses diverge light rays as shown in Fig.. 9.12 (b).. Such lenses are also
called diverging lenses.. A double concave lens is simply called a concave
lens.. A lens, either a convex lens or a concave lens, has two spherical
surfaces.. Each of these surfaces forms a part of a sphere.. The centres of
these spheres are called centres of curvature of the lens.. The centre of
curvature of a lens is usually represented by the letter C. Since there are two
centres of curvature, we may represent them a